In [14]:
import pandas as pd
#Quandl data sets (acess site to check)
import quandl
import math, datetime
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from matplotlib import style

import pickle

In [15]:
#import google stock df on Quandl
df = quandl.get('WIKI/GOOGL')

In [16]:
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [17]:
#just to exemplify, since stock value are more correlated to revenew, growth, etc
df = df[['Adj. Open','Adj. High','Adj. Low', 'Adj. Close', 'Adj. Volume']]

df['Hl_percentage'] = (df['Adj. High'] - df['Adj. Close'])/df['Adj. Close'] * 100
df['percent_change'] = (df['Adj. Close'] - df['Adj. Open'])/df['Adj. Open'] * 100

In [18]:
forecast_col = 'Adj. Close'
df.fillna(-99999, inplace=True)

forecast_out = int(math.ceil(0.01*len(df)))
                   
#to predict stock price of forecast out
df['label'] = df['Adj. Close'].shift(-forecast_out)
              
df.tail()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,Hl_percentage,percent_change,label
Date,,,,,,,,
2018-03-21,1092.57,1108.70,1087.21,1094.00,1990515.0,1.343693,0.130884,NaN
2018-03-22,1080.01,1083.92,1049.64,1053.15,3418154.0,2.921711,-2.487014,NaN
2018-03-23,1051.37,1066.78,1024.87,1026.55,2413517.0,3.918952,-2.360729,NaN
2018-03-26,1050.60,1059.27,1010.58,1054.09,3272409.0,0.491419,0.332191,NaN
2018-03-27,1063.90,1064.54,997.62,1006.94,2940957.0,5.720301,-5.353887,NaN


In [19]:
X = np.array(df.drop(['label'],1))
X = preprocessing.scale(X)

#set to predict
X_lately = X[-forecast_out:]

#set to train
X = X[:-forecast_out]

df.dropna(axis=0, how='any', inplace=True)

#results to train
y = np.array(df['label'])

3424
3389 3389


,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,Hl_percentage,percent_change,label
Date,,,,,,,,
2018-01-30,1177.72,1187.93,1174.51,1177.37,1792602.0,0.896914,-0.029718,1094.00
2018-01-31,1183.81,1186.32,1172.10,1182.22,1643877.0,0.346805,-0.134312,1053.15
2018-02-01,1175.99,1187.45,1169.36,1181.59,2774967.0,0.495942,0.476195,1026.55
2018-02-02,1127.42,1131.30,1111.17,1119.20,5798880.0,1.081129,-0.729098,1054.09
2018-02-05,1100.61,1114.99,1056.74,1068.76,3742469.0,4.325574,-2.893850,1006.94


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = LinearRegression()
#uses much possible processor
#clf = LinearRegression(njobs=-1)

#suport vector machine/regression
#clf = svm.SVR(kernel='poly')
#clf = svm.SVR(kernel='linear')

clf.fit(X_train, y_train)

#save classifier
with open('linearregression.pickle','wb') as f:
    pickle.dump(clf, f)
    
#load classifier
pickle_in = open('linearregression.pickle', 'rb')
clf = pickle.load(pickle_in)

accuracy = clf.score(X_test, y_test)
print(accuracy)

0.9800754042708563


In [ ]:
forecast_set = clf.predict(X_lately)

#new data to predict
print(forecast_set)

In [ ]:
#simple graphic stile
style.use('ggplot')

#column with nan
df['Forecast'] = np.nan

last_date = df.iloc[-1].name
last_unix = last_date.timestamp()

one_day = 86400
next_unix = last_unix + one_day

for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]
    
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()